In [64]:
#working from Google colab
#!pip install infomap

In [15]:
from infomap import Infomap
import pandas as pd
import numpy as np
import networkx as nx #will be using to visualizing inside the notebook
import matplotlib.pyplot as plt
import pathlib
np.random.seed(0)

In [65]:
#!infomap --help

In [52]:
trials = 7
im_1=Infomap()
#im.readInputData()
filename = 'num_freq_thr_1'
im_1.read_file('num_freq_thr_1', accumulate=False)

im_1.run(num_trials=trials)
#name = 'test'
codelength_1 = im_1.codelengths
print(
       f"Found {im_1.max_depth} levels with {im_1.num_leaf_modules} leaf modules in {im_1.num_top_modules} top modules and codelength: {im_1.codelength:.8f} bits"
 )
print(f"All codelengths:", codelength_1)

#print("Tree:\n# path node_id module_id flow")


# pathlib.Path("output").mkdir(exist_ok=True)
# print(f"Writing top level modules to output/{name}.clu...")
# im_1.write(f"output/{name}.clu")

# print(f"Writing second level modules to output/{name}_level2.clu...")
# im_1.write(f"output/{name}_level2.clu", depth_level=2)

# print(f"Writing bottom level modules to output/{name}_level-1.clu...")
# im_1.write(f"output/{name}_level-1.clu", depth_level=-1)

# print(f"Writing tree to output/{name}.tree...")
# im_1.write(f"output/{name}.tree")

# print("Read back .clu file and only calculate codelength...")
# im2 = Infomap(
#     silent=True, two_level=True, no_infomap=True, cluster_data=f"output/{name}.clu"
# )
# im2.read_file(filename)
# im2.run()
# print(
#     f"Found {im2.max_depth} levels with {im2.num_top_modules} top modules and codelength: {im2.codelength:.8f} bits"
# )

# print("Done!")

Found 3 levels with 38 leaf modules in 37 top modules and codelength: 5.43421352 bits
All codelengths: (5.437179508827913, 5.44118995997627, 5.437005634685901, 5.43452758253349, 5.434213522066828, 5.43452758253349, 5.440482157128201)


In [3]:
def runinfomap(trials,filename):

  im = Infomap()
  im.read_file(filename, accumulate=False)
  im.run(num_trials=trials)
  codelength = im.codelengths
  levels = im.max_depth
  modules = im.num_leaf_modules
  return codelength, levels, modules

In [27]:
trials = 5
filenames = []
codelengths = []
levels = []
modules = []
for i in range(0,16):
  filenames.append('num_freq_thr_'+str(i))

for f in filenames:
  codelength, level, module = runinfomap(trials,f)
  codelengths.append(codelength)
  levels.append(level)
  modules.append(module)


In [54]:
def cut_poll(filename,ID):
  df = pd.read_csv(filename,delimiter=' ', names = ['poll_ID', 'plant_ID', 'freq'])
  df_subset = df.loc[df['poll_ID'] == ID]
  df_new = df.merge(df_subset, how='left', indicator=True)
  df_new = df_new[df_new['_merge'] == 'left_only']
  df_new = df_new.drop(columns = ['_merge'])
  df = df_new
  df.to_csv('rm_pol'+str(ID)+'_'+str(filename),sep = ' ',index=False, header = False)
  return str('rm_pol'+str(ID)+'_'+str(filename))


In [55]:
def cut_plant(filename,ID):
  df = pd.read_csv(filename,delimiter=' ', names = ['poll_ID', 'plant_ID', 'freq'])
  df_subset = df.loc[df['poll_ID'] == ID]
  df_new = df.merge(df_subset, how='left', indicator=True)
  df_new = df_new[df_new['_merge'] == 'left_only']
  df_new = df_new.drop(columns = ['_merge'])
  df = df_new
  df.to_csv('rm_plant'+str(ID)+'_'+str(filename),sep = ' ',index=False,header = False)
  return str('rm_plant'+str(ID)+'_'+str(filename))

In [56]:
#take one random, one most frequent, one least frequent plant and pollinator to kill
df = pd.read_csv('num_freq_thr_0',delimiter=' ', names= ['poll_ID', 'plant_ID','freq'])
pol_IDS = df['poll_ID'].unique()
plant_IDS = df['plant_ID'].unique()
pol_rand_ID = np.random.choice(pol_IDS)
plant_rand_ID = np.random.choice(plant_IDS)
poll_max = df['poll_ID'].value_counts().idxmax()
poll_min = df['poll_ID'].value_counts().idxmin()
plant_max = df['plant_ID'].value_counts().idxmax()
plant_min = df['plant_ID'].value_counts().idxmin()
pols = [pol_rand_ID,poll_max,poll_min]
plants = [plant_rand_ID,plant_max, plant_min]


In [57]:
#create data files after killing
new_fnames_pols = [] #store the new filenames here
new_fnames_plants = []
for pol_ID in pols:
  new_fnames_pols.append(cut_poll("num_freq_thr_0",pol_ID))
for plant_ID in plants:
  new_fnames_plants.append(cut_plant("num_freq_thr_0",plant_ID))


In [58]:
codelengths_1 = [] #1 = killed pollinators, 2 = killed plants
codelengths_2 = []
levels_1 = []
levels_2 = []
modules_1 = []
modules_2 = []

for f_1 in new_fnames_pols:
  codelength, level, module = runinfomap(trials,f_1)
  codelengths_1.append(codelength)
  levels_1.append(level)
  modules_1.append(module)
for f_2 in new_fnames_plants:
  codelength, level, module = runinfomap(trials,f_2)
  codelengths_2.append(codelength)
  levels_2.append(level)
  modules_2.append(module)
#to-do: analysis with this data

In [22]:
#radnom tests not important
df_test1 = pd.read_csv("num_freq_thr_1",delimiter=' ', names = ['poll_ID', 'plant_ID', 'freq'])
df_test1
df_subset = df_test1.loc[df_test1['plant_ID'] ==df_test1['plant_ID'].value_counts().idxmax() ]

df_new = df_test1.merge(df_subset, how='left', indicator=True)
df_new = df_new[df_new['_merge'] == 'left_only']
df_new = df_new.drop(columns = ['_merge'])
df_test1 = df_new
df_subset.shape


(189, 3)